In [1]:
import numpy as np
import music21 as m21
from tqdm import tqdm

In [3]:
x = np.load('../data/augmented_and_padded_data_650_songs_with_measure_info.npz')

In [6]:
print(x.files)

['name', 'melody_pcps', 'chord_pcps', 'chord_roots', 'chord_types', 'measure_onsets']


In [7]:
names = x['name']
melody_pcps = x['melody_pcps']
chord_pcps = x['chord_pcps']
chord_roots = x['chord_roots']
chord_types = x['chord_types']
measure_onsets = x['measure_onsets']

In [11]:
print(melody_pcps.shape)
print(measure_onsets.shape)

(7800, 129, 12)
(7800, 68)


In [9]:
# key finding
def tonality_from_pcp( pcp ):
    major_profile = m21.analysis.discrete.KrumhanslSchmuckler().getWeights('major')
    minor_profile = m21.analysis.discrete.KrumhanslSchmuckler().getWeights('minor')
    major_corrs = np.zeros(12).astype(np.float32)
    minor_corrs = np.zeros(12).astype(np.float32)
    for i in range(12):
        major_corrs[i] = np.corrcoef( pcp, np.roll( 
            major_profile, i ) )[0][1]
        minor_corrs[i] = np.corrcoef( pcp, np.roll( 
            minor_profile, i ) )[0][1]
    major_max_idx = np.argmax( major_corrs )
    minor_max_idx = np.argmax( minor_corrs )
    major_max = np.max( major_corrs )
    minor_max = np.max( minor_corrs )
    if major_max > minor_max:
        return {'root': major_max_idx,
                'mode': 'major',
                'correlation': major_max}
    else:
        return {'root': minor_max_idx,
                'mode': 'minor',
                'correlation': minor_max}
# end tonality_from_pcp

In [12]:
# keep only c major pieces
c_melody_pcps = []
c_chord_pcps = []
c_measure_onsets = []

for i in tqdm(range(melody_pcps.shape[0])):
    tm = tonality_from_pcp( np.sum(melody_pcps[i,:,:], axis=0) )
    tc = tonality_from_pcp( np.sum(chord_pcps[i,:,:], axis=0) )
    if tm['root'] == 0 and tm['mode'] == 'major' and tm['correlation'] > 0.7 and \
        tc['root'] == 0 and tc['mode'] == 'major' and tc['correlation'] > 0.7:
        c_melody_pcps.append( melody_pcps[i,:,:] )
        c_chord_pcps.append( chord_pcps[i,:,:] )
        c_measure_onsets.append( measure_onsets[i,:] )

100%|██████████| 7800/7800 [00:27<00:00, 280.40it/s]


In [13]:
c_mel_np = np.array( c_melody_pcps )
print(c_mel_np.shape)
c_chr_np = np.array( c_chord_pcps )
print(c_chr_np.shape)
c_msr_np = np.array( c_measure_onsets )
print(c_msr_np.shape)

(104, 129, 12)
(104, 129, 12)
(104, 68)


In [ ]:
np.savez('../data/jazz_c_major_bars.npz', melody_pcps=c_mel_np, chord_pcps=c_chr_np, measure_onsets=c_msr_np)